# Loading CSVs to AWS Athena db

Our sensor data is a collection of CSV of over 30 million records.  For us to analyze this huge dataset, we will load them to Athena db.  [Athena](https://aws.amazon.com/athena/) is Amazon's interactive query service to analyze data in Amazon S3 using standard SQL.  I chose Athena because:
* **our data is already hosted in S3** - with minimal data structure we can use it directly for quary and analysis.  I also provide this data as a csv download for anyone, so the same s3 bucket can be used for sharing and distribution.
* **its serverless** - we don't need to deploy an sql server to query and analyze.  At the same time you only gets billed whenever the data is queried. This makes it attractctive as the cost is cheaper.

This guide walksthrough the basic setup of the Athena db using our weather data csv. I found a great introductory video about the basics of using Athena from [Academind](https://www.youtube.com/playlist?list=PL55RiY5tL51rZooHydslYclCYio7eoC66)

*Note: We will use mostly terminal CLI and Amazon's Athena Query UI, most of these commands can be executed with AWS [Command Line Interface](https://aws.amazon.com/cli/?sc_channel=PS&sc_campaign=acquisition_IN&sc_publisher=google&sc_medium=command_line_b&sc_content=aws_cli_e&sc_detail=aws%20cli&sc_category=command_line&sc_segment=159807028048&sc_matchtype=e&sc_country=IN&s_kwcid=AL!4422!3!159807028048!e!!g!!aws%20cli&ef_id=VgvD3AAAAAAp9jGB:20180729163045:s), but for simplicity, we will use the UI fromAWS console.*

### Preparing the data

```sh
head asg.csv 
id,name,time,value,water_level_change,time_difference,lon,lat,the_geom
1,iloilojalaurbridgepototan_wl,2013-09-03 04:00:00,12,1.39,160,122.686106,10.909531,0101000020E6100000B8E52329E9AB5E407765170CAED12540
2,isabelamalligbridge_rwl,2013-09-03 11:50:00,12.8,0,10,121.612111,17.1755,0101000020E610000066A19DD32C675E40B0726891ED2C3140
3,metromanilarosariojs_wl,2013-08-25 00:10:00,12.99,-0.0299999999999994,10,121.090436,14.597919,0101000020E6100000639813B4C9455E4065506D7022322D40
```

Our raw CSV contains several columns, for example, in the automated stream gauge (asg), we have:

* id - unique for each record
* name - name of station
* time - time of record
* value - water level in meters
* water_level_change - +/-  change of water level in meter
* time_difference - time difference from previous reading
* lon - longitude of station
* lat - latitude of station
* the_geom - WKB of the station location

* We remove the `the_geom` column since we don't need it for Athena, the `lat` and `lon` values is enough if we need basic spatial query later.

```sh
cut -d',' -f 1-8 asg.csv > asg-clean.csv
```

Then we compress the cleaned csv.

```sh
gzip arg-clean.csv
```

Repeat for all the other 3 files.

```sh
ls -alh *.csv *.gz
-rw-r--r--  1 maning  staff   243M Jul 28 15:13 arg-clean.csv.gz
-rw-r--r--  1 maning  staff   1.6G Jan 21  2017 arg.csv
-rw-r--r--  1 maning  staff   127M Jul 28 15:15 asg-clean.csv.gz
-rw-r--r--  1 maning  staff   936M Jan 21  2017 asg.csv
-rw-r--r--  1 maning  staff    40M Jul 28 15:17 aws-clean.csv.gz
-rw-r--r--  1 maning  staff   337M Jan 21  2017 aws.csv
-rw-r--r--  1 maning  staff   255M Jul 28 15:20 td-clean.csv.gz
-rw-r--r--  1 maning  staff   1.9G Jan 21  2017 td.csv
```

Compression is very important to reduce your cost.  Amazon charged you "based on the amount of data scanned by each query".  Compression reduces the file size and thus reducing the cost. For example by cleaning and compressing, the previous 4.7 GB of the combined 4 files is reduced to only 665 MB or about 7x smaller.

There are other data formats (i.e. Apache ORC, parquet) that can siginificantly reduce cost, but for simplicity we will use the compressed CSVs.

### Upload to S3 

For Athena to access the data, we need to upload the data to an s3 bucket.

* Go to S3 from the [AWS Console](https://s3.console.aws.amazon.com/s3/home?region=us-east-1#)
* Create a new bucket (for example, `s3://dostsensor-db/`)
* Create sub directories for each compressed csv.  Each of our csv has a different data structure. A subdirectory in the main bucket can be treated as a separate table in Athena. Use the following structure for your bucket:

```sh
tree
├──dostsensor-db
 ├── arg
 │   ├── arg-clean.csv.gz
 └── asg
 │   ├── arg-clean.csv.gz
 └── aws
 │   ├── aws-clean.csv.gz
 └── td
 │   ├── td-clean.csv.gz
    
```

### The AWS Athena UI

* To access Athena, go [Athena via AWS console](https://console.aws.amazon.com/athena/home?region=us-east-1).
* The Athena UI has the following parts:
  1. Main query editor - where you view/write your SQL code.
  2. List of available database and tables.
  3. Where SQL result are displaed.
  4. Tabs to other features of the UI.

![](img/athena-ui.png)

### Setup your database and tables

* From the Athena UI query window, create a new database.

```
CREATE DATABASE dostsensor;
```

Once created, the database will appear in the dropbox of the Database list.


* Defining the table's Data Definition Language (DDL).  Each table should have a DDL, this is the schema 
that Athena will use to read the CSV.  More info about Ahena's DDLs [here](https://docs.aws.amazon.com/athena/latest/ug/ddl-sql-reference.html).
* For the asg.csv, I used the ddl below:

```sql
CREATE EXTERNAL TABLE IF NOT EXISTS dostsensor.arg (
  `id` int, 
  `name` string, 
  `time` string, 
  `value` decimal(10,6), 
  `lon` decimal(10,6), 
  `lat` decimal(10,6)
  )
ROW FORMAT DELIMITED 
  FIELDS TERMINATED BY ',' 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  's3://dostsensor-db/asg'
TBLPROPERTIES (
  'has_encrypted_data'='false', 
  'skip.header.line.count'='1')
```

* [todo] - explain the parts of the DDL
* Once the table is created, it will appear in the list of tables under the `dostsensor` database
* Do the same for all your CSVs.

### Test your database

Once your tables are ready, we can test if the dat is displayed as expected.
* In the quaery editor, type

```
SELECT * FROM "dostsensor"."arg" limit 10;
```

* The result should display in the Result window

```
|   | id       | name                                | time                | value    | lon        | lat       |
| - | -------- | ----------------------------------- | ------------------- | -------- | ---------- | --------- |
| 1 | 32156535 | apayaoflora_r2                      | 2015-08-29 14:15:00 | 0.000000 | 121.471109 | 18.016230 |
| 2 | 32156538 | isabelacvaardcomplexisuechague_r1   | 2015-08-29 14:15:00 | 0.000000 | 121.683333 | 16.716667 |
| 3 | 32156539 | agusandelsurmagsaysayprosperidad_r2 | 2015-08-29 14:30:00 | 0.000000 | 126.007080 | 8.632080  |
| 4 | 32156540 | aklanguadalupebridgemadalag_rwl     | 2015-08-29 14:30:00 | 0.000000 | 122.309917 | 11.529853 |
```

Now all your CSV are available for query and analysis through Athena.  In the next section, we will discuss how to connect the Athena database to Jupyter notebook and pandas.

## See also
* [AWS Athena Guide](https://docs.aws.amazon.com/athena/latest/ug/what-is.html)
* [Athena videos by Academind](https://www.youtube.com/playlist?list=PL55RiY5tL51rZooHydslYclCYio7eoC66)